In [27]:
import os

# Add OpenAI library
import openai

# Get Configuration Settings
from dotenv import load_dotenv
load_dotenv()

True

In [28]:
# Configure OpenAI API using Azure OpenAI
openai.api_key = os.getenv("API_KEY")
openai.api_base = os.getenv("ENDPOINT")
openai.api_type = "azure"  # Necessary for using the OpenAI library with Azure OpenAI
openai.api_version = "2024-02-01"  # Latest / target version of the API

In [29]:
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader

In [30]:
# OpenAI Settings
model_deployment = "text-embedding-ada-002"
# SDK calls this "engine", but naming it "deployment_name" for clarity

model_name = "text-embedding-ada-002"

In [31]:
openai_embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
    openai_api_version = os.getenv("OPENAI_API_VERSION"), openai_api_key = os.getenv("API_KEY"),
    openai_api_base = os.getenv("ENDPOINT"), openai_api_type = "azure"
)

### Loading Junior and Senior Lesson Embeddings

In [32]:
from langchain_chroma import Chroma

In [33]:
# Elementary
vector_store_elementary = Chroma(
    collection_name="Elementary_Lessons",
    embedding_function=openai_embeddings,
    persist_directory="/Users/mac/Documents/Gospel-Companion/chroma_afc_sunday_school_lessons_db",  # Where to save data locally, remove if not neccesary
)


In [34]:
# Junior
vector_store_junior = Chroma(
    collection_name="Junior_Lessons",
    embedding_function=openai_embeddings,
    persist_directory="/Users/mac/Documents/Gospel-Companion/chroma_afc_sunday_school_lessons_db",  # Where to save data locally, remove if not neccesary
)


In [35]:
# Senior
vector_store_senior = Chroma(
    collection_name="Senior_Lessons",
    embedding_function=openai_embeddings,
    persist_directory="/Users/mac/Documents/Gospel-Companion/chroma_afc_sunday_school_lessons_db",  # Where to save data locally, remove if not neccesary
)

### Prompt Structure

In [36]:
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory, ConversationBufferWindowMemory
from langchain import PromptTemplate

In [37]:
# from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
# from langchain.chains import create_history_aware_retriever, create_retrieval_chain
# from langchain.chains.combine_documents import create_stuff_documents_chain

In [38]:
template_j = """
Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Junior Sunday school lessons".
------
<ctx>
{context}
</ctx>
------
<hs>
{history}
</hs>
------
{question}
Answer:
"""
prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template_j,
)

### Retrieval QA

In [39]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.llms import OpenAI

In [40]:
import langchain
langchain.verbose = True

In [41]:
llm = ChatOpenAI(temperature = 0.6, openai_api_key = os.getenv("API_KEY"), openai_api_base = os.getenv("ENDPOINT"), model_name="gpt-35-turbo", engine="Voicetask")

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


## Junior Lessons

### Similarity Search

In [42]:
retriever_j = vector_store_junior.as_retriever(search_kwargs={'k': 5})

In [43]:
qa_stuff = RetrievalQA.from_chain_type(
    llm = llm, 
    chain_type = "stuff", 
    retriever = retriever_j, 
    verbose = True,
    chain_type_kwargs = {
        "verbose": True,
        "prompt": prompt,
        "memory": ConversationBufferWindowMemory(
                k = 10,
                memory_key = "history",
                input_key = "question"),
                }
    )

In [49]:
results = vector_store_junior.similarity_search("What are the steps to creation? Answer in bullet form using the days and the creation", k=5)

for result in results:
    print(result)

page_content='1. How did the world come into existence?   How do we know?\n\n2. What did God do on the first day?\n\n3. What did God say at the close of every day of creation?\n\n4. Why did God create the world?\n\n5. Why does the ocean stay within its bounds?\n\n6. What different kinds of flesh are there?' metadata={'source': '/Users/mac/Documents/Gospel-Companion/data/Junior/Lesson1_Junior.txt'}
page_content='First, God spoke the heavens and the earth into existence, apparently a mass of substance, hidden in darkness.  Then He began to bring order out of that which was without form.  Thoughtfully He did all this for you and me.\n\nThe First Day\n\nOn the first day God made light.  There were no sun, moon, nor stars but God could make light without them, for He is light.  Jesus said,  "I am the light of the world" (John 8:12).  "That was the true Light, which lighteth every man that cometh into the world" (John 1:9).' metadata={'source': '/Users/mac/Documents/Gospel-Companion/data/Jun

In [44]:
query = "What are the steps to creation? Answer in bullet form using the days and the creation"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))

/Users/mac/Documents/Gospel-Companion/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Junior Sunday school lessons".
------
<ctx>
1. How did the world come into existence?   How do we know?

2. What did God do on the first day?

3. What did God say at the close of every day of creation?

4. Why did God create the world?

5. Why does the ocean stay within its bounds?

6. What different kinds of flesh are there?

First, God spoke the heavens and the earth into existence, apparently a mass of substance, hidden in darkness.  Then He began to bring order out of that which was without f

In [249]:
query = "What is the proof that God actually rested on the 7th day of creation"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Junior Sunday school lessons".
------
<ctx>
While God, with the completion of the creation, rested on the seventh day and hallowed it, yet it was not given to the Israelites to observe this day until the Law was written, when the seventh day was established as a Sabbath in commemoration of their deliverance out of bondage ([DEU:5:15]).  Jesus emphasised the necessity of Christians observing in spirit all the other laws given in the Ten Commandments but He never repeated the command to observe the

In [250]:
query = "What really is then the reason Christians worship on Sunday? And is this correct according to the bible as soon groups still say that the Sabbath day, as in the day for worship, Saturday."
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Junior Sunday school lessons".
------
<ctx>
The Lord's Day

One more instruction given in this lesson is to keep the Sabbath Day holy. One of the Ten Commandments given to the Children of Israel many years before the time of Isaiah was: "Remember the Sabbath day, to keep it holy" [EXO:20:8]). The first day of the week became accepted as the day of worship immediately after Jesus' resurrection, and has been observed by all true Christians from that time down to the present day. It is the day on wh

In [251]:
query = "What are some of the most controversial and debated doctrines of the Bible?"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Junior Sunday school lessons".
------
<ctx>
Christians are warned specifically against those who say we must keep the forms and ceremonies of the Law of Moses, whereas the Bible says that, "by the deeds of the law there shall no flesh be justified in his sight" [ROM:3:20]). We are warned against those who deny that Jesus is the Son of God who came to earth to die and rise again to save us from sin, whereas the Bible says, "Whosoever shall confess that Jesus is the Son of God, God dwelleth in him,


> Finished chain.

> Finished chain.

> Finished chain.
Some of the most controversial and debated doctrines of the Bible include the role of women in the church, the interpretation of the creation account in Genesis, the nature of the Trinity, the concept of predestination, the existence of Hell, and the interpretation of biblical prophecy.


In [252]:
query = "What about eternal Salvation, when one believes that once he is born again, he can continue in sin but he still remians born again."
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Junior Sunday school lessons".
------
<ctx>
Because a person is saved at one time, does not necessarily mean that he will always be saved. Unless he lives for Jesus and does not permit sin to enter his life he will lose his right to eternal life [EZE:18:26]). There is power in the Blood of Jesus to save, and to keep, if a person wants to be kept in the Lord.

Resisting Temptation

There are people today who feel that if they go to church on Sunday morning, and never do anyone any harm, they will go to Heaven. Others think that if the

In [253]:
query = "What can you tell me about the book of Hebrews that a new Christian should know?"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Junior Sunday school lessons".
------
<ctx>
Bible Text: [HEB:1:1-14].

Lesson 432 - Junior

Memory Verse: [HEB:1:1-14].

Notes"Go ye therefore, and teach all nations, baptizing them in the name of the Father, and of the Son, and of the Holy  Ghost" (Matthew 28:19).

The Sacrifices Fulfilled

The Book of Hebrews was written by the Apostle Paul to the Hebrew Christians, those who had believed on Jesus. They had been brought up to obey the Law of Moses, and they knew all about the sacrifices that sh

In [254]:
qa_stuff.combine_documents_chain.memory.load_memory_variables({})

{'history': "Human: What are the steps to creation? Answer in bullet form using the days and the creation\nAI: 1. God spoke the heavens and the earth into existence\n2. Day 1: God made light\n3. Day 2: God separated the waters and created the sky\n4. Day 3: God created dry land and vegetation\n5. Day 4: God created the sun, moon, and stars\n6. Day 5: God created sea creatures and birds\n7. Day 6: God created land animals and humans\n8. On the seventh day, God rested.\nHuman: What is the proof that God actually rested on the 7th day of creation\nAI: In the Junior Sunday school lessons, it is stated that God rested on the seventh day and hallowed it, but it was not until the Law was written that the Israelites were commanded to observe it as a Sabbath. There is no specific proof in the Bible that God rested on the 7th day of creation, but it is a part of the biblical narrative.\nHuman: What really is then the reason Christians worship on Sunday? And is this correct according to the bible

In [255]:
print(qa_stuff.combine_documents_chain.memory.buffer)

Human: What are the steps to creation? Answer in bullet form using the days and the creation
AI: 1. God spoke the heavens and the earth into existence
2. Day 1: God made light
3. Day 2: God separated the waters and created the sky
4. Day 3: God created dry land and vegetation
5. Day 4: God created the sun, moon, and stars
6. Day 5: God created sea creatures and birds
7. Day 6: God created land animals and humans
8. On the seventh day, God rested.
Human: What is the proof that God actually rested on the 7th day of creation
AI: In the Junior Sunday school lessons, it is stated that God rested on the seventh day and hallowed it, but it was not until the Law was written that the Israelites were commanded to observe it as a Sabbath. There is no specific proof in the Bible that God rested on the 7th day of creation, but it is a part of the biblical narrative.
Human: What really is then the reason Christians worship on Sunday? And is this correct according to the bible as soon groups still sa

### MMR

In [256]:
retriever_j = vector_store_junior.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 6, 'lambda_mult': 0.8}
)

In [257]:
qa_stuff = RetrievalQA.from_chain_type(
    llm = llm, 
    chain_type = "stuff", 
    retriever = retriever_j, 
    verbose = True,
    chain_type_kwargs = {
        "verbose": True,
        "prompt": prompt,
        "memory": ConversationBufferWindowMemory(
                k = 10,
                memory_key = "history",
                input_key = "question"),
                }
    )

In [258]:
query = "What are the steps to creation? Answer in bullet form using the days and the creation"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Junior Sunday school lessons".
------
<ctx>
1. How did the world come into existence?   How do we know?

2. What did God do on the first day?

3. What did God say at the close of every day of creation?

4. Why did God create the world?

5. Why does the ocean stay within its bounds?

6. What different kinds of flesh are there?

First, God spoke the heavens and the earth into existence, apparently a mass of substance, hidden in darkness.  Then He began to bring order out of that which was without form.  Thoughtfully He did all this for

In [259]:
query = "What is the proof that God actually rested on the 7th day of creation"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Junior Sunday school lessons".
------
<ctx>
While God, with the completion of the creation, rested on the seventh day and hallowed it, yet it was not given to the Israelites to observe this day until the Law was written, when the seventh day was established as a Sabbath in commemoration of their deliverance out of bondage ([DEU:5:15]).  Jesus emphasised the necessity of Christians observing in spirit all the other laws given in the Ten Commandments but He never repeated the command to observe the Sabbath.

The Day of Rest

God said H

In [260]:
query = "What really is then the reason Christians worship on Sunday? And is this correct according to the bible as soon groups still say that the Sabbath day, as in the day for worship, Saturday."
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Junior Sunday school lessons".
------
<ctx>
The Lord's Day

One more instruction given in this lesson is to keep the Sabbath Day holy. One of the Ten Commandments given to the Children of Israel many years before the time of Isaiah was: "Remember the Sabbath day, to keep it holy" [EXO:20:8]). The first day of the week became accepted as the day of worship immediately after Jesus' resurrection, and has been observed by all true Christians from that time down to the present day. It is the day on wh

In [261]:
query = "What are some of the most controversial and debated doctrines of the Bible?"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Junior Sunday school lessons".
------
<ctx>
Christians are warned specifically against those who say we must keep the forms and ceremonies of the Law of Moses, whereas the Bible says that, "by the deeds of the law there shall no flesh be justified in his sight" [ROM:3:20]). We are warned against those who deny that Jesus is the Son of God who came to earth to die and rise again to save us from sin, whereas the Bible says, "Whosoever shall confess that Jesus is the Son of God, God dwelleth in him,

In [262]:
query = "What about eternal Salvation, when one believes that once he is born again, he can continue in sin but he still remians born again."
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Junior Sunday school lessons".
------
<ctx>
Because a person is saved at one time, does not necessarily mean that he will always be saved. Unless he lives for Jesus and does not permit sin to enter his life he will lose his right to eternal life [EZE:18:26]). There is power in the Blood of Jesus to save, and to keep, if a person wants to be kept in the Lord.

Resisting Temptation

There are people today who feel that if they go to church on Sunday morning, and never do anyone any harm, they will 

In [263]:
query = "What can you tell me about the book of Hebrews that a new Christian should know?"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Junior Sunday school lessons".
------
<ctx>
Bible Text: [HEB:1:1-14].

Lesson 432 - Junior

Memory Verse: [HEB:1:1-14].

Notes"Go ye therefore, and teach all nations, baptizing them in the name of the Father, and of the Son, and of the Holy  Ghost" (Matthew 28:19).

The Sacrifices Fulfilled

The Book of Hebrews was written by the Apostle Paul to the Hebrew Christians, those who had believed on Jesus. They had been brought up to obey the Law of Moses, and they knew all about the sacrifices that sh

In [264]:
qa_stuff.combine_documents_chain.memory.load_memory_variables({})

{'history': 'Human: What are the steps to creation? Answer in bullet form using the days and the creation\nAI: - Day 1: God created light\n- Day 2: God divided the waters and created the sky\n- Day 3: God created dry land and vegetation\n- Day 4: God created the sun, moon, and stars\n- Day 5: God created sea creatures and birds\n- Day 6: God created land animals and humans\n- Day 7: God rested\nHuman: What is the proof that God actually rested on the 7th day of creation\nAI: The proof that God rested on the seventh day of creation is found in Genesis 2:2-3 in which it is stated that "And on the seventh day God ended his work which he had made; and he rested on the seventh day from all his work which he had made. And God blessed the seventh day, and sanctified it: because that in it he had rested from all his work which God created and made."\nHuman: What really is then the reason Christians worship on Sunday? And is this correct according to the bible as soon groups still say that the 

In [265]:
print(qa_stuff.combine_documents_chain.memory.buffer)

Human: What are the steps to creation? Answer in bullet form using the days and the creation
AI: - Day 1: God created light
- Day 2: God divided the waters and created the sky
- Day 3: God created dry land and vegetation
- Day 4: God created the sun, moon, and stars
- Day 5: God created sea creatures and birds
- Day 6: God created land animals and humans
- Day 7: God rested
Human: What is the proof that God actually rested on the 7th day of creation
AI: The proof that God rested on the seventh day of creation is found in Genesis 2:2-3 in which it is stated that "And on the seventh day God ended his work which he had made; and he rested on the seventh day from all his work which he had made. And God blessed the seventh day, and sanctified it: because that in it he had rested from all his work which God created and made."
Human: What really is then the reason Christians worship on Sunday? And is this correct according to the bible as soon groups still say that the Sabbath day, as in the 

## Senior Lessons

### Similarity Search

In [266]:
template_s = """
Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Senior Sunday school lessons".
------
<ctx>
{context}
</ctx>
------
<hs>
{history}
</hs>
------
{question}
Answer:
"""
prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template_s,
)

In [267]:
retriever_s = vector_store_senior.as_retriever(search_kwargs={'k': 5})

In [268]:
qa_stuff = RetrievalQA.from_chain_type(
    llm = llm, 
    chain_type = "stuff", 
    retriever = retriever_s, 
    verbose = True,
    chain_type_kwargs = {
        "verbose": True,
        "prompt": prompt,
        "memory": ConversationBufferWindowMemory(
                k = 10,
                memory_key = "history",
                input_key = "question"),
                }
    )

In [269]:
query = "What are the steps to creation? Answer in bullet form using the days and the creation"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Senior Sunday school lessons".
------
<ctx>
The steps of creation day by day are most wonderful. God's great purpose was to provide this world for man. In order that man should be sustained here in life, it was necessary that the earth be prepared for the creating of life. The successive creative acts reveal great wisdom and order. God first brought into being the earth, which was without form and void, and darkness was upon the face of the deep. His next act was to command light; for light and t

In [270]:
query = "What really is then the reason Christians worship on Sunday? And is this correct according to the bible as soon groups still say that the Sabbath day, as in the day for worship, Saturday."
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Senior Sunday school lessons".
------
<ctx>
In the Ten Commandments we are told to "remember the sabbath day, to keep it holy." Many people are seeking to accuse us regarding our observance of the Christian Sabbath, the first day of the week; but the fact that Jesus has honoured and blessed that day by His resurrection and His post-resurrection appearances to the assembled disciples, along with the fact that His Holy Spirit has been pleased to put His seal upon it as the day of worship during the

In [271]:
query = "What are some of the most controversial and debated doctrines of the Bible?"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Senior Sunday school lessons".
------
<ctx>
1. Who are some of the enemies of the Church Militant?

2. Why must there be no compromise with those who pervert the Word of God?

3. Name several ways by which to detect false prophets.

4. Why should a Christian not take away from, nor add to, the Word of God?

5. What warning is given in the Bible against preaching "another gospel”?

6. What is meant by the statement, "By their fruits ye shall know them”?

The Lord wants His people to show moderatio

In [272]:
query = "What about eternal Salvation, when one believes that once he is born again, he can continue in sin but he still remians born again."
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Senior Sunday school lessons".
------
<ctx>
Eternal Security

There is a false teaching in the world today that when once one has been born again he is always a child of God and will not be lost eternally regardless of how deep he may go into sin, and though he die in his sins. They call it "eternal security." Some say that people who have once been truly saved and have backslidden will eventually come back to God before they die, though they may wander in sin for a long time. It is true that the

In [273]:
query = "What can you tell me about the book of Hebrews that a new Christian should know?"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Senior Sunday school lessons".
------
<ctx>
Here is given us the secret of success in the Christian life: complete obedience.  One cannot be a Christian unless he obeys what God tells him to do.  One cannot continue in God's favour if he refuses to walk in the will of God and rebels at the command of God.  Obedience is the keystone of it all, the one requirement which, if we carefully heed, will not only bring us to the "borders of the land" but will take us on over that we might enjoy the blessi

In [274]:
qa_stuff.combine_documents_chain.memory.load_memory_variables({})

{'history': 'Human: What are the steps to creation? Answer in bullet form using the days and the creation\nAI: I. The Work of Creation on the First and Second Days\n- God created Heaven and earth\n- The earth was without form and void\n- God commanded light\n- Night and day were ordained\n- God made the firmament to divide the waters\n- The waters below were divided from the waters above\n- The firmament God called Heaven\n\nII. The Work of Creation on the Third and Fourth Days\n- The waters below were gathered into one place, and the dry land appeared\n- The land God called Earth, and the gathering together of the waters, Seas\n- Grass and fruits were brought forth on the earth\n- God placed lights in the firmament\n- God ordained the greater light to rule the day, and the lesser light to rule the night; He made the stars also\n\nIII. The Work of Creation on the Fifth and Sixth Days\n- God created moving creatures to inhabit the waters, and fowls to inhabit the firmament\n- God made e

In [275]:
print(qa_stuff.combine_documents_chain.memory.buffer)

Human: What are the steps to creation? Answer in bullet form using the days and the creation
AI: I. The Work of Creation on the First and Second Days
- God created Heaven and earth
- The earth was without form and void
- God commanded light
- Night and day were ordained
- God made the firmament to divide the waters
- The waters below were divided from the waters above
- The firmament God called Heaven

II. The Work of Creation on the Third and Fourth Days
- The waters below were gathered into one place, and the dry land appeared
- The land God called Earth, and the gathering together of the waters, Seas
- Grass and fruits were brought forth on the earth
- God placed lights in the firmament
- God ordained the greater light to rule the day, and the lesser light to rule the night; He made the stars also

III. The Work of Creation on the Fifth and Sixth Days
- God created moving creatures to inhabit the waters, and fowls to inhabit the firmament
- God made every living creature and every w

### MMR

In [282]:
retriever_s = vector_store_senior.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 6, 'lambda_mult': 0.8}
)

In [283]:
qa_stuff = RetrievalQA.from_chain_type(
    llm = llm, 
    chain_type = "stuff", 
    retriever = retriever_s, 
    verbose = True,
    chain_type_kwargs = {
        "verbose": True,
        "prompt": prompt,
        "memory": ConversationBufferWindowMemory(
                k = 10,
                memory_key = "history",
                input_key = "question"),
                }
    )

In [284]:
query = "What are the steps to creation? Answer in bullet form using the days and the creation"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Senior Sunday school lessons".
------
<ctx>
The steps of creation day by day are most wonderful. God's great purpose was to provide this world for man. In order that man should be sustained here in life, it was necessary that the earth be prepared for the creating of life. The successive creative acts reveal great wisdom and order. God first brought into being the earth, which was without form and void, and darkness was upon the face of the deep. His next act was to command light; for light and t

In [285]:
query = "What is the proof that God actually rested on the 7th day of creation"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Senior Sunday school lessons".
------
<ctx>
Now he is going back to the first type of this rest, the rest that God took after He completed the work of creation. Upon that type was built all the Sabbaths of the entire Israelitish religious program, which God gave them. They had the weekly sabbaths; they had the Sabbath of Weeks that was later called Pentecost. They had the monthly Sabbaths every seventh month, which was the Feast of Tabernacles. They had the Sabbath of years: every seventh year wa

In [286]:
query = "What really is then the reason Christians worship on Sunday? And is this correct according to the bible as soon groups still say that the Sabbath day, as in the day for worship, Saturday."
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Senior Sunday school lessons".
------
<ctx>
In the Ten Commandments we are told to "remember the sabbath day, to keep it holy." Many people are seeking to accuse us regarding our observance of the Christian Sabbath, the first day of the week; but the fact that Jesus has honoured and blessed that day by His resurrection and His post-resurrection appearances to the assembled disciples, along with the fact that His Holy Spirit has been pleased to put His seal upon it as the day of worship during the

In [287]:
query = "What are some of the most controversial and debated doctrines of the Bible?"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Senior Sunday school lessons".
------
<ctx>
1. Who are some of the enemies of the Church Militant?

2. Why must there be no compromise with those who pervert the Word of God?

3. Name several ways by which to detect false prophets.

4. Why should a Christian not take away from, nor add to, the Word of God?

5. What warning is given in the Bible against preaching "another gospel”?

6. What is meant by the statement, "By their fruits ye shall know them”?

The Lord wants His people to show moderatio

In [288]:
query = "What about eternal Salvation, when one believes that once he is born again, he can continue in sin but he still remians born again."
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Senior Sunday school lessons".
------
<ctx>
Eternal Security

There is a false teaching in the world today that when once one has been born again he is always a child of God and will not be lost eternally regardless of how deep he may go into sin, and though he die in his sins. They call it "eternal security." Some say that people who have once been truly saved and have backslidden will eventually come back to God before they die, though they may wander in sin for a long time. It is true that the

In [289]:
query = "What can you tell me about the book of Hebrews that a new Christian should know?"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Senior Sunday school lessons".
------
<ctx>
Faith comes to our hearts by believing and obeying the Word of God.  The eleventh chapter of Hebrews gives us a list of God's heroes who were able to accomplish great things for God because they had faith.  It was strength and support to them in the time of their great need, sore trial, bereavement, or persecution.  It was the guiding star that brought them to their spiritual land of promise, and caused them to see the heavenly City above and beyond all

In [290]:
qa_stuff.combine_documents_chain.memory.load_memory_variables({})

{'history': 'Human: What are the steps to creation? Answer in bullet form using the days and the creation\nAI: 1. Day 1: God created light and separated it from darkness.\n2. Day 2: God created the firmament to divide the waters above from the waters below.\n3. Day 3: God gathered the waters below into one place, making the dry land appear. He also created grass, fruits, and herbs with seed after their kind.\n4. Day 4: God created the sun, moon, and stars to divide light from darkness and to be signs for seasons, days, and years.\n5. Day 5: God created sea creatures and birds.\n6. Day 6: God created land animals, including cattle, beasts, and creeping things, and finally created man in His own image.\nHuman: What is the proof that God actually rested on the 7th day of creation\nAI: The proof that God actually rested on the 7th day of creation is found in Genesis 2:2-3 of the King James Version of the Bible, which states, "And on the seventh day God ended his work which he had made; and

In [291]:
print(qa_stuff.combine_documents_chain.memory.buffer)

Human: What are the steps to creation? Answer in bullet form using the days and the creation
AI: 1. Day 1: God created light and separated it from darkness.
2. Day 2: God created the firmament to divide the waters above from the waters below.
3. Day 3: God gathered the waters below into one place, making the dry land appear. He also created grass, fruits, and herbs with seed after their kind.
4. Day 4: God created the sun, moon, and stars to divide light from darkness and to be signs for seasons, days, and years.
5. Day 5: God created sea creatures and birds.
6. Day 6: God created land animals, including cattle, beasts, and creeping things, and finally created man in His own image.
Human: What is the proof that God actually rested on the 7th day of creation
AI: The proof that God actually rested on the 7th day of creation is found in Genesis 2:2-3 of the King James Version of the Bible, which states, "And on the seventh day God ended his work which he had made; and he rested on the sev

## Elementary

### Similarity Search

In [296]:
template_e = """
Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Elementary Sunday school lessons".
------
<ctx>
{context}
</ctx>
------
<hs>
{history}
</hs>
------
{question}
Answer:
"""
prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template_e,
)

In [297]:
retriever_s = vector_store_elementary.as_retriever(search_kwargs={'k': 5})
qa_stuff = RetrievalQA.from_chain_type(
    llm = llm, 
    chain_type = "stuff", 
    retriever = retriever_s, 
    verbose = True,
    chain_type_kwargs = {
        "verbose": True,
        "prompt": prompt,
        "memory": ConversationBufferWindowMemory(
                k = 10,
                memory_key = "history",
                input_key = "question"),
                }
    )

In [298]:
query = "What are the steps to creation? Answer in bullet form using the days and the creation"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Elementary Sunday school lessons".
------
<ctx>
Questions


1.  Who made the Heaven and the earth and all the things that are in them?  (Genesis 1:1).

2.  How did God make all these things?

3.  What did God make in the first day? (Genesis 1:3).

4.  Who were with God the Father when He made all these things?

5.  In whose likeness did God make man? (Genesis 1:26).

God Made the Sky


God, on the second day, made a beautiful sky and the clouds that carry water to make the flowers and trees and p

In [299]:
query = "What really is then the reason Christians worship on Sunday? And is this correct according to the bible as soon groups still say that the Sabbath day, as in the day for worship, Saturday."
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Elementary Sunday school lessons".
------
<ctx>
All this happened the first day of the week, or Sunday, and is called the Lord's Day. Ever since that time people who believe and love the whole Bible have kept that day as a special day of worship, instead of Saturday, which the Children of Israel kept.


Telling the World


As Jesus talked with them, He told them He had come to earth to show men how to live, and had been willing to die on the cross, and had risen again so that everyone might be sa

In [300]:
query = "What are some of the most controversial and debated doctrines of the Bible?"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Elementary Sunday school lessons".
------
<ctx>
Questions


1. Who made the heaven and the earth? [GEN:1:1].


2. Did Adam and Eve disobey God? [GEN:3:6].


3. Why did Jesus Christ come to earth? [JHN:3:16].


4. Did Jesus promise to come back to earth some day? [JHN:14:3].


5. Where is the river of water of life? [REV:22:1].

Bible Text: [GEN:2:7-17]; [GEN:3:1-6]; [JHN:1:1-5]; [JHN:3:16]; [JHN:14:1-3]; [REV:22:1-5].


        
                  [GEN:2:7-17]; [GEN:3:1-6]; [JHN:1:1-5]; [JHN:3:16]

In [301]:
query = "What about eternal Salvation, when one believes that once he is born again, he can continue in sin but he still remians born again."
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Elementary Sunday school lessons".
------
<ctx>
Jesus gives one a heart that wants to please Him instead of wanting to disobey Him. When God has made that change in a heart then one is "born again.” God wants all men to be saved – to be "born again.” And Jesus came to earth to tell people how they could be born again, and He did everything He could to help them to have their sins forgiven and be made ready for their home in Heaven with Jesus.


Questions


1. What must we do to be ready for Heave

In [302]:
query = "What can you tell me about the book of Hebrews that a new Christian should know?"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Elementary Sunday school lessons".
------
<ctx>
-– not the Antioch from where we started, but another city by the same name. We make our way through rugged mountain passes where there are many dangers. Robbers may be hiding along the trail, but God's angels are watching over us, and nothing can harm us.


Now it is the Sabbath Day, the day the Jews go to their church, called a synagogue. We go to this church and sit among the Jews who are gathered for a meeting. Listen! Someone is reading from th

In [303]:
qa_stuff.combine_documents_chain.memory.load_memory_variables({})

{'history': "Human: What are the steps to creation? Answer in bullet form using the days and the creation\nAI: 1. God made light (Genesis 1:3-5)\n2. God made the sky and clouds (Genesis 1:6-8)\n3. God made the land and water (Genesis 1:9-10)\n4. God made the sun, moon, and stars (Genesis 1:14-19)\n5. God created living creatures in the water and sky (Genesis 1:20-23)\n6. God created living creatures on land (Genesis 1:24-25)\n7. God made man in His own image (Genesis 1:26-31)\nHuman: What really is then the reason Christians worship on Sunday? And is this correct according to the bible as soon groups still say that the Sabbath day, as in the day for worship, Saturday.\nAI: According to the Elementary Sunday school lessons, Christians worship on Sunday because it is the Lord's Day, the day when Jesus rose from the grave. It has been kept as a special day of worship by those who believe and love the whole Bible. However, there are still some groups that worship on Saturday, the Sabbath d

In [304]:
print(qa_stuff.combine_documents_chain.memory.buffer)

Human: What are the steps to creation? Answer in bullet form using the days and the creation
AI: 1. God made light (Genesis 1:3-5)
2. God made the sky and clouds (Genesis 1:6-8)
3. God made the land and water (Genesis 1:9-10)
4. God made the sun, moon, and stars (Genesis 1:14-19)
5. God created living creatures in the water and sky (Genesis 1:20-23)
6. God created living creatures on land (Genesis 1:24-25)
7. God made man in His own image (Genesis 1:26-31)
Human: What really is then the reason Christians worship on Sunday? And is this correct according to the bible as soon groups still say that the Sabbath day, as in the day for worship, Saturday.
AI: According to the Elementary Sunday school lessons, Christians worship on Sunday because it is the Lord's Day, the day when Jesus rose from the grave. It has been kept as a special day of worship by those who believe and love the whole Bible. However, there are still some groups that worship on Saturday, the Sabbath day. While the Bible do

### MMR

In [306]:
retriever_e = vector_store_elementary.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 6, 'lambda_mult': 0.8}
)

In [307]:
qa_stuff = RetrievalQA.from_chain_type(
    llm = llm, 
    chain_type = "stuff", 
    retriever = retriever_e, 
    verbose = True,
    chain_type_kwargs = {
        "verbose": True,
        "prompt": prompt,
        "memory": ConversationBufferWindowMemory(
                k = 10,
                memory_key = "history",
                input_key = "question"),
                }
    )

In [308]:
query = "What are the steps to creation? Answer in bullet form using the days and the creation"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Elementary Sunday school lessons".
------
<ctx>
Questions


1.  Who made the Heaven and the earth and all the things that are in them?  (Genesis 1:1).

2.  How did God make all these things?

3.  What did God make in the first day? (Genesis 1:3).

4.  Who were with God the Father when He made all these things?

5.  In whose likeness did God make man? (Genesis 1:26).

God Made the Sky


God, on the second day, made a beautiful sky and the clouds that carry water to make the flowers and trees and p

In [309]:
query = "What is the proof that God actually rested on the 7th day of creation"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Elementary Sunday school lessons".
------
<ctx>
Everything God Made Was Good


God saw that everything He had made was very good; and then He rested on the seventh day and blessed it.


The Beautiful Garden


God prepared a beautiful garden for Adam and for those who would be with him. He gave them trees that had good fruit, lovely flowers for them to enjoy, vegetables for their food, and many other good things. He told Adam that he might eat of all the fruit except from one of the trees. If they

In [310]:
query = "What really is then the reason Christians worship on Sunday? And is this correct according to the bible as soon groups still say that the Sabbath day, as in the day for worship, Saturday."
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Elementary Sunday school lessons".
------
<ctx>
All this happened the first day of the week, or Sunday, and is called the Lord's Day. Ever since that time people who believe and love the whole Bible have kept that day as a special day of worship, instead of Saturday, which the Children of Israel kept.


Telling the World


As Jesus talked with them, He told them He had come to earth to show men how to live, and had been willing to die on the cross, and had risen again so that everyone might be sa

In [311]:
query = "What are some of the most controversial and debated doctrines of the Bible?"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Elementary Sunday school lessons".
------
<ctx>
Questions


1. Who made the heaven and the earth? [GEN:1:1].


2. Did Adam and Eve disobey God? [GEN:3:6].


3. Why did Jesus Christ come to earth? [JHN:3:16].


4. Did Jesus promise to come back to earth some day? [JHN:14:3].


5. Where is the river of water of life? [REV:22:1].

Bible Text: [GEN:2:7-17]; [GEN:3:1-6]; [JHN:1:1-5]; [JHN:3:16]; [JHN:14:1-3]; [REV:22:1-5].


        
                  [GEN:2:7-17]; [GEN:3:1-6]; [JHN:1:1-5]; [JHN:3:16]

In [312]:
query = "What about eternal Salvation, when one believes that once he is born again, he can continue in sin but he still remians born again."
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Elementary Sunday school lessons".
------
<ctx>
Jesus gives one a heart that wants to please Him instead of wanting to disobey Him. When God has made that change in a heart then one is "born again.” God wants all men to be saved – to be "born again.” And Jesus came to earth to tell people how they could be born again, and He did everything He could to help them to have their sins forgiven and be made ready for their home in Heaven with Jesus.


Questions


1. What must we do to be ready for Heave

In [313]:
query = "What can you tell me about the book of Hebrews that a new Christian should know?"
# response = qa_stuff.run(query)
print(qa_stuff.run(query))



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following context (delimited by <ctx></ctx>) and the chat history (delimited by <hs></hs>) to answer the user's question. 
All bible texts are to referenced with King James Version. You can give info on bible texts.
If you don't know the answer, just say that you don't know, don't try to make up an answer. Refer to the context provided as "the Elementary Sunday school lessons".
------
<ctx>
-– not the Antioch from where we started, but another city by the same name. We make our way through rugged mountain passes where there are many dangers. Robbers may be hiding along the trail, but God's angels are watching over us, and nothing can harm us.


Now it is the Sabbath Day, the day the Jews go to their church, called a synagogue. We go to this church and sit among the Jews who are gathered for a meeting. Listen! Someone is reading from th

In [314]:
qa_stuff.combine_documents_chain.memory.load_memory_variables({})

{'history': 'Human: What are the steps to creation? Answer in bullet form using the days and the creation\nAI: 1. God made light (Day 1)\n2. God made the sky (Day 2)\n3. God made the land and waters (Day 3)\n4. God made the lights (sun, moon, stars) (Day 4)\n5. God put living things in the world (fish, birds) (Day 5)\n6. God made man in His own image (Day 6)\nHuman: What is the proof that God actually rested on the 7th day of creation\nAI: The proof that God rested on the 7th day of creation is found in Genesis 2:2-3, which says "And on the seventh day God ended his work which he had made; and he rested on the seventh day from all his work which he had made. And God blessed the seventh day, and sanctified it: because that in it he had rested from all his work which God created and made."\nHuman: What really is then the reason Christians worship on Sunday? And is this correct according to the bible as soon groups still say that the Sabbath day, as in the day for worship, Saturday.\nAI: 

In [315]:
print(qa_stuff.combine_documents_chain.memory.buffer)

Human: What are the steps to creation? Answer in bullet form using the days and the creation
AI: 1. God made light (Day 1)
2. God made the sky (Day 2)
3. God made the land and waters (Day 3)
4. God made the lights (sun, moon, stars) (Day 4)
5. God put living things in the world (fish, birds) (Day 5)
6. God made man in His own image (Day 6)
Human: What is the proof that God actually rested on the 7th day of creation
AI: The proof that God rested on the 7th day of creation is found in Genesis 2:2-3, which says "And on the seventh day God ended his work which he had made; and he rested on the seventh day from all his work which he had made. And God blessed the seventh day, and sanctified it: because that in it he had rested from all his work which God created and made."
Human: What really is then the reason Christians worship on Sunday? And is this correct according to the bible as soon groups still say that the Sabbath day, as in the day for worship, Saturday.
AI: According to the Elemen